In [21]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Reading the Dataset
dataset = pd.read_csv('CKD.csv')
dataset = pd.get_dummies(dataset,drop_first=True)
dataset.shape
independ= dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
                   'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
                   'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_yes', 'pe_yes', 'ane_yes']]
depend = dataset[['classification_yes']]

#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independ, depend, test_size = 1/3, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']}

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

# fitting the model for grid search 
grid.fit(X_train, y_train) 

grid_predictions = grid.predict(X_test) # Pass test set for prediction

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)


# classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

from sklearn.metrics import f1_score
# {'weighted', 'samples', 'binary', 'micro', 'macro'} 
f1_micro=f1_score(y_test,grid_predictions,average='weighted')

from sklearn.metrics import roc_auc_score
roc = roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

cv_result=grid.cv_results_

print("\nLogistic Grid Classifiction Report:\n")
print("The confusion_matrix value:\n\n {}".format(cm))
print("The classification_report value:\n\n {}".format(clf_report))
print("The f1_micro value : {}\n".format(f1_micro))
print("The roc_auc_score value for best parameter {}: {}\n".format(grid.best_params_, roc))

# df = pd.DataFrame(cv_result)
# df
# Convert DataFrame to markdown table
# table = df.to_markdown(index=False)
# print(table)

Fitting 5 folds for each of 4 candidates, totalling 20 fits

Logistic Grid Classifiction Report:

The confusion_matrix value:

 [[51  0]
 [ 1 81]]
The classification_report value:

               precision    recall  f1-score   support

       False       0.98      1.00      0.99        51
        True       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133

The f1_micro value : 0.9924946382275899

The roc_auc_score value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 1.0



C:\Users\ranja\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
import pickle # import pickle
file_name = "finalized_model_logistic_classification.sav" #file name
pickle.dump(grid,open(file_name,'wb')) # open the file and use 'wb' write/save the model
# loaded_model = pickle.load(open(file_name,'rb')) # open the file and use 'rb' read the model then load the loaded variable
# loaded_model.predict([[123456,246,876,0,1]]) # pass the custom input value